1. Import thư viện

- bs4: BeautifulSoup để parse HTML
- selenium: để tạo cơ hội cho JS chạy và load dữ liệu cho trang web
- pandas: để lưu dữ liệu vào dataframe và xuất dữ liệu


In [4]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager


2. Setup webdriver và xác định url cần lấy dữ liệu


In [5]:
service = Service(executable_path=EdgeChromiumDriverManager().install())

url_stats = 'https://www.timeshighereducation.com/world-university-rankings/2023/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats'

url_score = 'https://www.timeshighereducation.com/world-university-rankings/2023/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/scores'


3. Load trang web và thu thập và xử lí code HTML
   Hai ô dưới đây thực hiện thực hiện quy trình tương tự nhau:

- Load trang web bằng `webdriver` vừa tạo
- Thu thập code HTML của web
- Sử dụng `BeautifulSoup` để `parse` HTML
- Sử dụng `find_all` để lấy được tất cả các đối tượng mà ta đã xác định là cần thiết

Nhận ra rằng nếu thực hiện click vào tên trường đại học, ta sẽ được dẫn tới một trang mới có thêm thông tin về trường đại học đó.

Ý tưởng sử dụng những đường link này sẽ là làm một mảng lưu các link này sẽ được sử dụng ở bước tiếp theo.


In [8]:
web_driver_stats = webdriver.ChromiumEdge(service=service)
web_driver_stats.get(url_stats)

# Đợi cho đến khi tất cả các phần tử được tải
web_driver_stats.implicitly_wait(10)

# Lấy code HTML của trang web
stats_html = web_driver_stats.page_source

# Parse HTML
stats_soup = bs(stats_html, 'html.parser')

# Lấy các đối tượng cần thiết
uni_stats_rank = stats_soup.find_all(
    'td', {'class': 'rank sorting_1 sorting_2'})
uni_name = stats_soup.find_all(name=["a", "div"], attrs={
                               "class": "ranking-institution-title"})
uni_location = stats_soup.findAll("div", {"class": "location"})
uni_stats_number_students = stats_soup.findAll(
    "td", {"class": "stats stats_number_students"})
uni_stats_students_per_staff = stats_soup.findAll(
    "td", {"class": "stats stats_student_staff_ratio"})
uni_stats_international_students = stats_soup.findAll(
    "td", {"class": "stats stats_pc_intl_students"})
uni_stats_female_male_ratio = stats_soup.findAll(
    "td", {"class": "stats stats_female_male_ratio"})

# Đóng trình duyệt
web_driver_stats.close()


In [9]:
# Score
web_driver_score = webdriver.ChromiumEdge(service=service)

web_driver_score.get(url_score)

# Đợi cho đến khi tất cả các phần tử được tải
web_driver_score.implicitly_wait(10)

# Lấy code HTML của trang web
score_html = web_driver_score.page_source

# Parse HTML
score_soup = bs(score_html, 'html.parser')

# Lấy các đối tượng cần thiết
# Vì thứ tự của các trường giống với `rank` nên ta chỉ cần bắt đầu lấy từ cột thứ 3
uni_overall_score = score_soup.findAll("td", {"class": "scores overall-score"})
uni_teaching_score = score_soup.findAll(
    "td", {"class": "scores teaching-score"})
uni_research_score = score_soup.findAll(
    "td", {"class": "scores research-score"})
uni_citations_score = score_soup.findAll(
    "td", {"class": "scores citations-score"})
uni_industry_income_score = score_soup.findAll(
    "td", {"class": "scores industry_income-score"})
uni_international_outlook_score = score_soup.findAll(
    "td", {"class": "scores international_outlook-score"})

# Đóng trình duyệt
web_driver_score.close()


4. Tạo dataframe để lưu dữ liệu


In [10]:
rank, names, countries, number_students, student_staff_ratio, intl_students, female_male_ratio, web_address = [
], [], [], [], [], [], [], []

overall_score, teaching_score, research_score, citations_score, industry_income_score, international_outlook_score = [], [], [], [], [], []

for i in range(len(uni_name)):
    names.append(uni_name[i].text)
    rank.append(uni_stats_rank[i].text)
    countries.append(uni_location[i].text)
    number_students.append(uni_stats_number_students[i].text)
    student_staff_ratio.append(uni_stats_students_per_staff[i].text)
    intl_students.append(uni_stats_international_students[i].text)
    female_male_ratio.append(uni_stats_female_male_ratio[i].text[0:2])
    overall_score.append(uni_overall_score[i].text)
    teaching_score.append(uni_teaching_score[i].text)
    research_score.append(uni_research_score[i].text)
    citations_score.append(uni_citations_score[i].text)
    industry_income_score.append(uni_industry_income_score[i].text)
    international_outlook_score.append(uni_international_outlook_score[i].text)


In [11]:
df = pd.DataFrame({
    'rank': rank,
    'name': names,
    'country': countries,
    'number_of_students': number_students,
    'student_per_staff': student_staff_ratio,
    'international_students': intl_students,
    'famale_male_ratio': female_male_ratio,
    'overall_score': overall_score,
    'teaching_score': teaching_score,
    'research_score': research_score,
    'citations_score': citations_score,
    'industry_income_score': industry_income_score,
    'international_outlook_score': international_outlook_score
})
df


,rank,name,country,number_of_students,student_per_staff,international_students,famale_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50,95.2,94.8,99.0,99.3,49.5,80.5
2,=3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47,94.8,90.9,99.5,97.0,54.2,95.8
3,=3,Stanford University,United States,"16,164",7.1,24%,46,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40,94.2,90.7,93.6,99.8,90.9,89.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2340,Reporter,York St John University,United Kingdom,"6,315",18.6,12%,65,n/a,n/a,n/a,n/a,n/a,n/a
2341,Reporter,"Yusuf Maitama Sule University, Kano",Nigeria,"12,880",33.0,0%,48,n/a,n/a,n/a,n/a,n/a,n/a
2342,Reporter,Zhytomyr Polytechnic State University,Ukraine,"3,869",15.4,1%,34,n/a,n/a,n/a,n/a,n/a,n/a
2343,Reporter,Ziauddin University,Pakistan,"4,906",8.8,1%,63,n/a,n/a,n/a,n/a,n/a,n/a


5. Làm sạch dữ liệu


Dễ nhận thấy có nhiều trường đại học có giá trị cột rank là `Reporter` thay vì một số cụ thể nào đó. Theo một bài đăng từ chính `timeshighereducation.com` vào tháng 9/2021 mà bạn có thể đọc ở [đây](https://www.timeshighereducation.com/world-university-rankings/why-we-are-including-reporter-institutions-our-world-university-rankings) thì:

```text
[...] so this year we have created a new “reporter” status for them. This status indicates that they are active participants in the rankings process, even though they have not been ranked this year.

Practically, this means that they are listed, alphabetically, at the end of the rankings table, and rather than being assigned a rank number, they are instead labelled reporter institutions.
```

Dịch bởi GG Translate:

```text
[...] vì vậy năm nay chúng tôi đã tạo một trạng thái "báo cáo" mới cho họ. Trạng thái này chỉ ra rằng đó là những  tham gia tích cực trong quá trình xếp hạng, ngay cả khi họ không được xếp hạng trong năm nay.
Trên thực tế, điều này có nghĩa là chúng được liệt kê, theo thứ tự bảng chữ cái, ở cuối bảng xếp hạng và thay vì được gán một số thứ hạng, thay vào đó chúng được dán nhãn là các tổ chức báo cáo.
```

Vì vậy, ta sẽ loại bỏ các giá trị `Reporter` trong cột `rank` và chỉ giữ lại các giá trị còn lại

Tiếp đó, ta sẽ loại bỏ kí tự `%` để có thể chuyển cột `international_students` thành kiểu dữ liệu số.

Dễ nhận thấy cột `Rank` có một số kiểu giá trị như sau:

- `=3` (có dấu `=`) là giá trị của các trường đại học có cùng rank
- `101-150` (có dấu `-`) là giá trị của các trường đại học có cùng rank từ 101 đến 150
- `1501+` (có dấu `+`) là giá trị của các trường đại học có cùng rank từ 1501 trở lên

Ta sẽ loại bỏ các kí tự `=`, và `+`, riêng những giá trị có `-` ta sẽ chỉ lấy tới trước dấu `-` để có thể chuyển cột `rank` thành kiểu dữ liệu số.

Điều này cũng xảy ra ở cột `Overall_score`.

Và cuối cùng là sự xuất hiện của các giá trị `n/a` trong các cột, ta cần thay thế chúng bằng giá trị NaN của thư viện `numpy`.


In [12]:
# loại bỏ các hàng có cột rank là 'Reporter'
df = df[df['rank'] != 'Reporter']
# loại bỏ kí tự '%' trong cột 'International Students'
df.loc[:, 'international_students'] = df.loc[:,
                                             'international_students'].str.replace(pat='%', repl='')
# loại bỏ kí tự `=` trong cột 'Rank'
df.loc[:, 'rank'] = df.loc[:, 'rank'].str.replace(pat='=', repl='')
# loại bỏ kí tự `-` và những kí tự trước đó của cột overall score
df.loc[:, 'overall_score'] = df.loc[:, 'overall_score'].str.replace(
    pat='.*\–', repl='', regex=True)
# loại bỏ kí tự `-` và những kí tự sau đó của cột 'Rank' và loại bỏ kí tự `+`
df.loc[:, 'rank'] = df.loc[:, 'rank'].str.replace(
    pat='\–\d*|\+', repl='', regex=True)
# bỏ dấu `,` trong cột 'Number of Students'
df.loc[:, 'number_of_students'] = df.loc[:,
                                         'number_of_students'].str.replace(pat=',', repl='')
# thay thế giá trị `n/a` trong mọi cột thành NaN
df = df.replace('n/a*', np.nan, regex=True)
df


,rank,name,country,number_of_students,student_per_staff,international_students,famale_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
0,1,University of Oxford,United Kingdom,20965,10.6,42,48,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,21887,9.6,25,50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,20185,11.3,39,47,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,16164,7.1,24,46,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,11415,8.2,33,40,94.2,90.7,93.6,99.8,90.9,89.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1501,Yamaguchi University,Japan,10724,11.1,7,38,18.3,21.3,12.3,13.9,43.8,30.4
1795,1501,Yanshan University,China,38649,13.2,1,39,18.3,17.2,10.9,18.3,46.9,17.0
1796,1501,Yeditepe University,Turkey,17038,28.2,5,55,18.3,18.8,12.2,10.5,65.7,29.6
1797,1501,Yuri Gagarin State Technical University of Sar...,Russian Federation,28280,34.1,1,42,18.3,12.5,8.9,7.2,37.5,16.7


6. Xuất dữ liệu


In [ ]:
# xuất dữ liệu ra file csv
df.to_csv('src/data/world_university_ranking_2023.csv', index=False)
